In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

# Chemin vers ChromeDriver
chrome_driver_path = 'C:/Users/user/Desktop/chromedriver-win64/chromedriver.exe'

# Options de Chrome
options = Options()
# options.add_argument("--headless")  # Décommentez pour exécuter en arrière-plan sans ouvrir une fenêtre Chrome
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Lancer le service
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

# Accéder à la page Web of Science
driver.get('https://www.webofscience.com/')

# Attendre que la page soit complètement chargée
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))
)

# Cliquer sur le lien "Sign in"
try:
    sign_in_link = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.LINK_TEXT, 'Sign in'))
    )
    sign_in_link.click()
    print("Lien 'Sign in' cliqué avec succès.")
except TimeoutException:
    print("Erreur : le lien 'Sign in' n'a pas pu être cliqué dans le temps imparti.")

# Saisir l'email
email_field = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[formcontrolname="email"]'))
)
email_field.send_keys('fadwatayanne18@gmail.com')
print("Email entré.")

# Saisir le mot de passe
password_field = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[formcontrolname="password"]'))
)
password_field.send_keys('UbVu\\X((-x9@ELc')
print("Mot de passe entré.")
password_field.send_keys(Keys.RETURN)

# Ajouter une vérification alternative pour la connexion réussie
try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.header-panel'))  # Mettez à jour avec un sélecteur valide
    )
    print("Connexion réussie.")
except TimeoutException:
    print("Le panneau d'en-tête n'a pas pu être trouvé. Vérification avec un autre élément.")

    # Vérifier si l'utilisateur est redirigé vers une page avec un autre élément confirmant la connexion
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="some-other-class"]'))  # Changez cela en fonction de la page
        )
        print("Connexion réussie (vérification alternative).")
    except TimeoutException:
        print("Échec de la vérification alternative de la connexion.")


Lien 'Sign in' cliqué avec succès.
Email entré.
Mot de passe entré.
Connexion réussie.


In [60]:
def rechercher_auteur(driver, last_name, first_name):
    try:
        # Attendre que les champs de recherche soient disponibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="mat-input-0"]'))  # Last name input
        )

        # Saisir le nom de famille
        last_name_input = driver.find_element(By.XPATH, '//*[@id="mat-input-0"]')
        last_name_input.send_keys(last_name)

        # Saisir le prénom
        first_name_input = driver.find_element(By.XPATH, '//*[@id="mat-input-1"]')
        first_name_input.send_keys(first_name)

        # Attendre que le bouton de recherche soit cliquable
        search_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="snSearchType"]/div[2]/button[2]'))
        )
        search_button.click()

        # Attendre un peu pour voir les résultats
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))  # Mettez à jour avec un sélecteur valide pour les résultats
        )

        print(f"Recherche effectuée avec succès pour : {first_name} {last_name}")

    except Exception as e:
        print(f"Erreur lors de la recherche de l'auteur : {e}")


In [61]:
# Après vous être connecté avec succès
rechercher_auteur(driver, "HAFIDI", "IMAD")


Recherche effectuée avec succès pour : IMAD HAFIDI


In [76]:
def extraire_informations_auteur(driver):
    infos_auteur = {}

    identifiant_auteur = driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-author-page/div/div/div[1]/app-author-record-header/div/app-author-details/div/div/div/span[1]').text
    infos_auteur['Identifiers'] = identifiant_auteur

    nom_complet = driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-author-page/div/div/div[1]/app-author-record-header/div/app-author-details/div/app-display-data[1]/div/div[2]/span').text
    infos_auteur['Published names'] = nom_complet

    pays_affiliation = driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-author-page/div/div/div[1]/app-author-record-header/div/app-author-details/div/app-display-data[2]/div/div[2]/div/div[1]/span').text
    infos_auteur['Organizations'] = pays_affiliation

    h_index = driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-author-page/div/div/div[2]/app-metrics-column/div/div[2]/app-citation-network/mat-card/mat-card-content/app-publication-metrics/div[2]/div[1]/div[1]').text
    infos_auteur['H-Index'] = h_index

    citations_totales = driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-author-page/div/div/div[2]/app-metrics-column/div/div[2]/app-citation-network/mat-card/mat-card-content/app-publication-metrics/div[2]/div[3]/div[1]').text
    infos_auteur['Sum of Times Cited'] = citations_totales

   
    return infos_auteur

In [79]:
 # Appel de la fonction pour extraire les informations de l'auteur
infos_auteur = extraire_informations_auteur(driver)
    
    # Vérification du résultat
if infos_auteur:
        print("Informations de l'auteur extraites avec succès :")
        for key, value in infos_auteur.items():
            print(f"{key} : {value}")
else:
        print("Aucune information d'auteur trouvée.")


Informations de l'auteur extraites avec succès :
Identifiers : Web of Science ResearcherID
Published names : Hafidi, Imad Hafidi, I.
Organizations : Sultan Moulay Slimane University of Beni Mellal
H-Index : 4
Sum of Times Cited : 42


In [81]:
def extraire_documents_auteur(driver):
    # Liste pour stocker les informations des documents
    documents_auteur = []

    try:
        # Attendre que la liste des documents soit chargée
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[13]/div/div/div[3]'))  # Ajustez si nécessaire
        )

        # Trouver tous les documents dans la liste
        document_elements = driver.find_elements(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/app-summary-title/h3')

        for doc in document_elements:
            info_document = {}
            # Extraire les auteurs
            auteurs = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/app-summary-authors/div/span[6]').text.strip()
            info_document['By'] = auteurs

            # Extraire le titre de l'article
            titre_article = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/app-summary-title/h3/a').text.strip()
            info_document['Title'] = titre_article

            # Extraire l'année de publication
            annee_publication = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/div/span[1]').text.strip()
            info_document['Published'] = annee_publication

            # Extraire le titre de la source
            titre_source = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/div/app-jcr-sidenav/mat-sidenav-container/mat-sidenav-content/span/a/span').text.strip()
            info_document['Source'] = titre_source

            # Extraire le nombre de citations
            nombre_citations = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[4]/div/div/div[1]/label').text.strip()
            info_document['Nombre de citations'] = nombre_citations

          

           

            # Ajouter le document aux informations de l'auteur
            documents_auteur.append(info_document)

        return documents_auteur

    except Exception as e:
        print(f"Erreur lors de l'extraction des documents : {e}")
        return None


In [83]:
  # Appel de la fonction pour extraire les informations des documents
documents = extraire_documents_auteur(driver)

    # Vérifier et afficher les informations des documents extraits
if documents:
        print("Documents extraits :")
        for doc in documents:
            print(f"Auteurs : {doc['By']}")
            print(f"Titre : {doc['Title']}")
            print(f"Année de publication : {doc['Published']}")
            print(f"Titre de la source : {doc['Source']}")
            print(f"Nombre de citations : {doc['Nombre de citations']}")
            print('-' * 40)  # Séparateur pour chaque document
else:
        print("Aucun document trouvé.")

Documents extraits :
Auteurs : Jamali, A
Titre : A robust approach for solving a vehicle routing problem with time windows with uncertain service and travel times
Année de publication : Win 2020
Titre de la source : INTERNATIONAL JOURNAL OF INDUSTRIAL ENGINEERING COMPUTATIONS
Nombre de citations : 14
Citations
----------------------------------------


In [86]:
def extraire_documents_auteur(driver):
    # Liste pour stocker les informations des documents
    documents_auteur = []

    try:
        # Attendre que la liste des documents soit chargée
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[13]/div/div/div[3]'))  # Ajustez si nécessaire
        )

        # Trouver tous les documents dans la liste
        document_elements = driver.find_elements(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/app-summary-title/h3')

        for doc in document_elements:
            info_document = {}
            # Extraire les auteurs
            auteurs = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/app-summary-authors/div/span[6]').text.strip()
            info_document['By'] = auteurs

            # Extraire le titre de l'article
            titre_article = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/app-summary-title/h3/a').text.strip()
            info_document['Title'] = titre_article

            # Extraire l'année de publication
            annee_publication = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/div/span[1]').text.strip()
            info_document['Published'] = annee_publication

            # Extraire le titre de la source
            titre_source = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[2]/div[1]/div/app-jcr-sidenav/mat-sidenav-container/mat-sidenav-content/span/a/span').text.strip()
            info_document['Source'] = titre_source

            # Extraire le nombre de citations
            nombre_citations = doc.find_element(By.XPATH, '//*[@id="mat-tab-content-0-0"]/div/app-publications-tab/div/app-publications-placeholder/app-author-document-summary/div/div/div/div/div[2]/app-records-list/app-record[2]/div/div/div[4]/div/div/div[1]/label').text.strip()
            info_document['Nombre de citations'] = nombre_citations

              # Extraire le DOI (s'il existe)
            try:
                doi = doc.find_element(By.XPATH, '//*[@id="snMainArticle"]/div[6]/span/div/span/div').text.strip()
                info_document['DOI'] = doi
            except:
                info_document['DOI'] = 'Non disponible'


           # Extraire le résumé (s'il existe)
            try:
                resume = doc.find_element(By.XPATH, '//*[@id="snMainArticle"]/div[8]/div').text.strip()
                info_document['Abstract'] = resume
            except:
                info_document['Abstract'] = 'Non disponible'

            # Extraire les mots-clés (s'ils existent)
            try:
                mots_cles = doc.find_element(By.XPATH, '//*[@id="snMainArticle"]/app-full-record-keywords/div').text.strip()
                info_document['Keywords'] = mots_cles
            except:
                info_document['Keywords'] = 'Non disponible'

            # Extraire le type de document (article, conférence, etc.)
            try:
                type_document = doc.find_element(By.XPATH, '//*[@id="snMainArticle"]/div[7]/div[4]').text.strip()
                info_document['Document Type'] = type_document
            except:
                info_document['Document Type'] = 'Non disponible'

          

            # Ajouter le document aux informations de l'auteur
            documents_auteur.append(info_document)

        return documents_auteur

    except Exception as e:
        print(f"Erreur lors de l'extraction des documents : {e}")
        return None


In [87]:

# Exemple d'utilisation : Appel de la fonction et affichage des résultats
documents = extraire_documents_auteur(driver)

# Afficher les documents extraits
if documents:
    print("Documents extraits :")
    for doc in documents:
        print(f"Titre : {doc['Title']}")
        print(f"Auteurs : {doc['By']}")
        print(f"Année de publication : {doc['Published']}")
        print(f"Titre de la source : {doc['Source']}")
        print(f"Nombre de citations : {doc['Nombre de citations']}")
        print(f"DOI : {doc['DOI']}")
        print(f"Résumé : {doc['Abstract']}")
        print(f"Mots-clés : {doc['Keywords']}")
        print(f"Type de document : {doc['Document Type']}")
        print('-' * 40)
else:
    print("Aucun document trouvé.")

Documents extraits :
Titre : A robust approach for solving a vehicle routing problem with time windows with uncertain service and travel times
Auteurs : Jamali, A
Année de publication : Win 2020
Titre de la source : INTERNATIONAL JOURNAL OF INDUSTRIAL ENGINEERING COMPUTATIONS
Nombre de citations : 14
Citations
DOI : Non disponible
Résumé : Non disponible
Mots-clés : Non disponible
Type de document : Non disponible
----------------------------------------
